In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pickle
from pathlib import Path

In [ ]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi

In [ ]:
username = 'marzahl'
password = '****'
host =  "https://exact.cs.fau.de/"
inference_results = Path('D:/Datasets/EIPH WSI/Humane/inference_results_boxes_all_human.p')

In [ ]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)

image_sets = image_sets_api.list_image_sets(name__contains="-Algorithm")
images = {}
annotation_types = {}

In [ ]:
for image_set in image_sets.results:
    for image in images_api.list_images(pagination=False, image_set=image_set.id).results:
        if len(image.annotations) == 0:
            images[image.name] = image
    for product in image_set.product_set:
        for anno_type in annotation_types_api.list_annotation_types(product=product).results:
            annotation_types[anno_type.name] = anno_type

In [ ]:
resultsArchive = pickle.load(open(str(inference_results),'rb'))

In [ ]:
threads = []
for image_name in images:
    image = images[image_name]

    file_annotations = resultsArchive[image.name][resultsArchive[image.name][:,5].argsort()]
    for idx in range(len(file_annotations)):
        row = file_annotations[idx]
        
        x_min = int(row[0]) 
        y_min = int(row[1]) 
        x_max = int(row[2]) 
        y_max = int(row[3])
        label = int(row[4]) 

        if x_max - x_min < 10 or y_max - y_min < 10:
            continue

        vector = {"x1": x_min, "y1": y_min, "x2": x_max, "y2": y_max}
        annotation_type = annotation_types[str(label)]

        annotation = Annotation(annotation_type=annotation_type.id, vector=vector, image=image.id)
        thread = annotations_api.create_annotation(body=annotation, async_req=True)
        threads.append(thread)

In [ ]:
from time import sleep
while (len(threads) > 0):
    for thread in threads:
        if thread.ready():
            threads.remove(thread)
    sleep(0.25)